# Semantics Similarity
This notebook generates multiple representations of the difference in semantic similarity between two datasets.

More specifically, it was designed to probe the semantic difference between specific categories of TruthfulQA and custom datasets created as *holdouts* for those categories.

The two methods are:
1. [Uniform Manifold Approximation and Projection](https://umap-learn.readthedocs.io/en/latest/index.html) (UMAP) to reduce dimensionality of embeddings and plot a visual representation. This is used as a sanity check; if we can percieve clear differences in the visual representation of the our `retro` and the `target`, our other test should probably pick up on these differences as well.
2. This one is slightly more complicated, but allows for a concrete p-value, and has shown to be more sensitive to differences than more straight forward methods, such as simply comparing mean cosine similarity.
    - First, we randomly sample the `combined` datasets for some numer of entries `s`; we designate the `i`th sampling `pulls[i]`. Note that `s` cannot be larger than the smallest of the two input datasets.
    - We then calculate the unique, nontrivial cosine similarities between `pulls[i]` and `combined`.
        - This excludes two types of entries: when an embedding is compared with itself, and when an embedding has already been compared (i.e., we do not compare A🠖B when we already have calculated B🠖A)
    - Leverage the [Anderson Darling k-sample test](https://www.itl.nist.gov/div898/software/dataplot/refman1/auxillar/andeksam.htm) (ADk) to create a *likelihood of same distribution* value for each sample, `pulls[i]`, when compared to the *combined dataset*.
        - This can be written `ADk_test_statistic(cos_sim(combined), cos_sim(pulls[i]))`.
    - We then use ADk to compare `combined` and `retro` in the same manner.
    - Finally, we calculate the likelihood that an ADk test statistic greater than the value seen when comparing `combined` and `retro` could be produced by replacing `retro` with a random sample.
        - If this is very likely (>95%) or very unlikely (<5%), we say that the datasets are not sufficiently similar, according to this test.


When run in Colab using free instances it takes ~5 minutes to run in its entirety.

In [ ]:
import time
t0 = time.time()

try:
  import google.colab
  IN_COLAB = True
  !git clone https://github.com/cwenner/retrospective-llm-eval.git
except:
  IN_COLAB = False
  import os
  import sys
  sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))


In [ ]:
!pip install -U sentence-transformers
!pip install datasets==2.15.0
!pip install scipy
!pip install umap-learn
!pip install umap-learn[plot]
!pip install umap-learn[parametric_umap]

In [ ]:
import datasets
import numpy as np
import scipy as sp
import sentence_transformers
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import umap
import seaborn as sns
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

# Prep
- Set user defined variables
- Choose our embedding model
- Create helper functions to load our datasets
- Create helper functions to generate the embeddings for our  datasets

In [ ]:
# DATASET_FILENAME = "crafted_dataset_unfiltered.jsonl"
DATASET_FILENAME = "crafted_nora+vasco_v1-gram.csv"
VERBOSE: bool = True

#### Choose Embedding Model

In [ ]:
embedding_model = sentence_transformers.SentenceTransformer(
    "all-mpnet-base-v2", device="cpu"
)

#### Helper Functions: Embedding

In [ ]:
# Encode a string using the chosen embedding model
    # NOTE: this action is described by the verb "to embed" in rest of notebook
def embed(text: str) -> np.array:
    return embedding_model.encode(text, convert_to_tensor=True).numpy() # @Cenny, why don't we use this function later in the "get_embed_mat" function

In [ ]:
def get_embed_mat(
    dataset_to_embed: datasets.Dataset,
    exclude_choices: bool = False,
) -> np.array:
    """
    Embed elements from a dataset that uses the TruthfulQA structure.

    Args:
        truthfulqa_dataset (datasets.Dataset): The dataset to embed.
        exclude_choices (bool, optional): If this is True, only the
            questions will be embedded. If this is False, the questions
            and choices will be embedded. Defaults to False.
    """
    if exclude_choices:
        texts = dataset_to_embed["question"]
    else:
        texts = [
            # "\n".join([x["question"]] + x["mc1_targets"]["choices"])
            "\n".join([x["question"]] + sorted(x["mc1_targets"]["choices"]))
            for x in dataset_to_embed
        ]

    if VERBOSE: list(map(lambda i: print(f'{i}\n{texts[i]}\n'), np.random.choice(len(texts),2)))

    return embedding_model.encode(texts)

#### Helper Functions: Datasets Loading

In [ ]:
# The HuggingFace (HF) TruthfulQA (TQA) dataset only has category labels in the
#   "generation" task version, but we also need the possible answers that are
#   found in the "multiple_choice" version. This function uses the categories
#   defined in the "generation" version to filter the "multiple_choice" version
    # NOTE: This probably isn't necessary for most other datasets
def load_truthfulqa(category=None):
    ds = datasets.load_dataset("truthful_qa", "multiple_choice")["validation"]
    if category and category != "all":
        filtered_ds = datasets.load_dataset("truthful_qa", "generation")
        filtered_ds = filtered_ds.filter(
            lambda x: x["category"].lower() == category.lower()
        )
        filtered_questions = filtered_ds["validation"]["question"]
        ds = ds.filter(lambda x: x["question"] in filtered_questions)
        if VERBOSE: print(f'TruthfulQA category loaded: {category}')
    return ds

In [ ]:
def custom_loader(ds):
    # Splits string by "." and sets 'ext' to the final substring;
    #   this is effectively the same a taking the file's extension
    *f, ext = DATASET_FILENAME.split(".")
    ext = ext.lower()

    # Switch statement for csv and json/jsonl files
    #   csv files need added logic to put them into the HF dataset format
    if ext == 'csv':
        def array(x, dtype=None):
            return x

        # Special logic due to how the CSV stores choices as a string
        import_ds = datasets.load_dataset(
            "csv", data_files=f'retrospective-llm-eval/data/datasets/{ds}'
        )["train"]

        import_ds = import_ds.map(
            lambda x: {
                "question": x["question"],
                "mc1_targets": eval(x["mc1_targets"], dict(globals(), array=array), locals()),
            }
        )

        # import_ds = import_ds.map(lambda x:
        #     dict(
        #         question=x["Rewritten in style"],
        #         mc1_targets=dict(
        #             choices=[
        #                 x
        #                 for x in [
        #                         x["Correct"],
        #                 ] + [
        #                     x[f"Incorrect{i}"]
        #                     for i in range(1, 11)
        #                 ]
        #                 if x
        #             ]
        #         )
        #     ),
        #     remove_columns=import_ds.column_names
        # )
    elif ext == 'jsonl'or ext == 'json':
        import_ds = datasets.load_dataset(
            "json", data_files=f'retrospective-llm-eval/data/datasets/{ds}'
        )["train"]
    return import_ds

#### Helper Functions: Write to Logfile

In [ ]:
def logfile_write(dataset_filename, content):
    *f, ext = dataset_filename.split(".")
    logfile = open(f'{f[0]}.txt','a')
    logfile.writelines(content)
    logfile.write('\n')
    logfile.close()

# Get Embeddings

## Datasets
``` load_truthfulqa ``` has special logic for only pulling specific categories of the dataset

``` custom_loader ``` can handle {.json; .jsonl; .csv}

In [ ]:
# target_ds = load_truthfulqa("misconceptions")

target_ds = load_truthfulqa("misconceptions")
retro_ds = custom_loader(DATASET_FILENAME)

## Caculate embeddings
Embeddings are calculated for the entire dataset entry, including both question and potential responses. Additional prompting (scaffolding) is not included in this embedding.

In [ ]:
# Get embeddings for both the target and retro datasets
target_embs = get_embed_mat(target_ds)
retro_embs = get_embed_mat(retro_ds)
num_entries = [embs.shape[0] for embs in [target_ds, retro_ds]]
if VERBOSE: print(f'Entries in target = {num_entries[0]}\nEntries in retro  = {num_entries[1]}\n')

# Stack the two matrices of embedded entries on top of eachother
combined_embs = np.concatenate((target_embs, retro_embs), axis = 0)
if VERBOSE: print(f'Shape of embedded combined datasets = {combined_embs.shape}')

# UMAP

In [ ]:
# Set up UMAP
reducer = umap.UMAP()
scaled_embs = StandardScaler().fit_transform(combined_embs)
umapped = reducer.fit_transform(scaled_embs)

In [ ]:
# Plot results of UMAP
fig, ax = plt.subplots(1,1,figsize=(8,8))

ax.scatter(umapped[:num_entries[0], 0],umapped[:num_entries[0], 1])
ax.scatter(umapped[num_entries[0]:, 0],umapped[num_entries[0]:, 1])
# ax.scatter(umapped[num_entries[0]:num_entries[0]+num_entries[1]-1, 0],umapped[num_entries[0]:num_entries[0]+num_entries[1]-1, 1])
# ax.scatter(umapped[num_entries[0]+num_entries[1]:, 0],umapped[num_entries[0]+num_entries[1]:, 1])
ax.legend(['Target', 'Retro'])
ax.set_aspect('equal', 'datalim')
plt.show()

# Original Semantics Similarity Test

#### Helper Functions: Cosine Similarity

In [ ]:
def self_sim_mat(embs: np.array) -> float:
    # INPUT: NxA matrix of N embbeddings; each row is a different prompt
    # OUTPUT: NxN masked matrix w/ the diagonal & upper diagonal masked;
        # each row (i) is the cosine similarity between the
        # (i)th embedding and each (other) embedding in the input matrix;
        # the entry corresponding to cosine similarity with itself is masked
    sims = sentence_transformers.util.cos_sim(embs, embs).numpy()
    mask = np.tril(np.ones(sims.shape[0]),0)
    return np.ma.masked_array(sims,mask)

In [ ]:
def cross_sim_vec(embs1: np.array, embs2: np.array) -> float:
    # INPUT:
        # embs1 ~ NxA matrix of N embbeddings; each row is a different prompt
        # embs2 ~ MxA matrix of M embbeddings; each row is a different prompt
    # OUTPUT: N*Mx1 vector which is the 1-dimensional representation of the
        # cosine similarities between embeddings (rows) in embs1 and embs2;
        # note that sims is an NxM matrix representation
    sims = sentence_transformers.util.cos_sim(embs1, embs2).numpy()
    return sims.flatten()

In [ ]:
def shuffle_pull(embs: np.array, samp_size: int, num_samples: int):
    # INPUTS:
        # embs ~ NxA matrix of N embbeddings; each row is a different prompt
        # samp_size ~ the number of rows to draw for each sample (dimension 2 of output matrix)
        # num_samples ~ number of samples to take (dimension 1 of output matrix)
    # OUTPUT 1: (num_samples)x(samp_size)x(embs.shape[1]);
        # each layer (constant value in dimension 1) is a set of embeddings for
        # (sample_size) random entries
    # OUTPUT 2: (num_samples)x(NUM REMAINING ENTRIES)x(embs.shape[1]);
        # each layer (constant value in dimension 1) is a set of embeddings for
        # the remaining entries in Combined
    pulls = np.ma.zeros((num_samples, samp_size, embs.shape[1]),dtype='float32')
    lefts = np.ma.zeros((num_samples, abs(samp_size-embs.shape[0]), embs.shape[1]),dtype='float32')
    for i in range(num_samples):
        np.random.shuffle(embs)
        pulls[i] = embs[:samp_size,:]
        lefts[i] = embs[samp_size:,:]
    return pulls, lefts

## Precompute All Cosine Similarities

In [ ]:
# Calculate the self similarities of the combined dataset
    # NOTE: We mask the diagonal because these entries represent comparing an
    #   embedding with itself; we also mask the upper triangle because it is a
    #   mirror of the values in the lower triangle.
combined_cos_sim = self_sim_mat(combined_embs).compressed()
entry_count = sum(num_entries)

## Anderson Darling k-Sample Test

#### Precompute all sub-samples

In [ ]:
# Number of samples (layers of the final matrix; times to run the AK test)
N = 25000

# Sample size of each layer (number of entries to pull for each sample)
s = min(num_entries)

# Actually generate the stack
pull_stack, left_stack = shuffle_pull(combined_embs, s, N)

# Print info
if VERBOSE: print(f'Number of times to run AK test = {N}\nNumber of samples per AK test = {s}')

#### Calculate AK statistics
For each sample, calculate AK statistic for whether the cosine similarities between the sample and the target could have been pulled from the same distribution as the cosine similarities of the combined embeddings.

For each sample, calculate AK statistic for whether the cosine similarities between the sample and itself could have been pulled from the same distribution as the cosine similarities of the combined embeddings.

In [ ]:
# Calculate the AK test statistic for determining whether each random sampling of
# embeddings could have been pulled from the distribution in the COMBINED dataset
    # shorthand: random_samples(COMBINED,len(RETRO))🠖COMBINED
ak_stats = np.zeros(N,)
for i in range(N):
    ak_stats[i] = sp.stats.anderson_ksamp([combined_cos_sim,
                                           np.concatenate((cross_sim_vec(left_stack[i], pull_stack[i]),
                                                           self_sim_mat(pull_stack[i]).compressed()))]).statistic

# # One-line version, doesn't seem to speed up anything though :/
# ak_stats = list(map(lambda i: sp.stats.anderson_ksamp([combined_cos_sim, np.concatenate((cross_sim_vec(left_stack[i], pull_stack[i]), self_sim_mat(pull_stack[i]).compressed()))]).statistic]).statistic, range(N)))

## Analysis of self-similarity distribution

---



In [ ]:
plt.hist(self_sim_mat(retro_embs).compressed(), bins=np.arange(-0.1, 1.0, 0.1), alpha=0.5, density=True, color='blue')
plt.hist(self_sim_mat(target_embs[:retro_embs.shape[0],:]).compressed(), bins=np.arange(-0.1, 1.0, 0.1), alpha=0.5, density=True,color='red')
plt.show()

In [ ]:
num_to_show = 10
print("Most similar pairs in each dataset:")
for ds_name, ds, embs_set in [("target", target_ds, target_embs), ("retro", retro_ds, retro_embs)]:
  print(f"Dataset: {ds_name}")
  # Compare sets of the same size
  embs_set = embs_set[:retro_embs.shape[0],:]

  self_sims_set = self_sim_mat(embs_set)
  for idx in np.argsort(-self_sims_set, axis=None)[:num_to_show]:
    x, y = np.unravel_index(idx, self_sims_set.shape)
    print(self_sims_set[x][y])
    print(ds[int(x)]["question"][:100])
    print(ds[int(y)]["question"][:100])
  print("----")

  # np.ma.indices(self_sim_mat(target_embs)

In [ ]:
# target_ds[63]["question"], target_ds[63]["question"]
# np.unravel_index(, target_self_sims.shape)

In [ ]:
# target_self_sims.shape

Conduct the same two tests, but this time use the retro set itself. Effectively, we are asking two questions:
1. How likely is it that the cosine similarities between the retro and the combined dataset were pulled from the same distribution as the cosine similarities between the combined dataset and itself?

2. How likely is it that the cosine similarities between the retro dataset and itself were pulled from the same distribution as the cosine similarities between the combined dataset and itself?


In [ ]:
retro_stat = sp.stats.anderson_ksamp([combined_cos_sim,
                                      np.concatenate((cross_sim_vec(target_embs, retro_embs),
                                                      self_sim_mat(retro_embs).compressed()))]).statistic
logfile_write(DATASET_FILENAME,f'Similarity ADk-test statistic for Retro: {retro_stat:.3f}')
print(f'Similarity ADk-test statistic for Retro: {retro_stat:.3f}')

## Plotting

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(11,9))

# LEFT PLOT: histogram of AK(random_samples(COMBINED,len(RETRO))🠖COMBINED);
# orange line is AK(Retro🠖COMBINED)
xstart = -2
counts, bins = np.histogram(ak_stats)
ymax = 1.02*max(counts)
ax.stairs(counts, bins, fill=True, alpha=.6)
ax.plot([retro_stat, retro_stat], [0, ymax])
ax.set_xlim(xstart, max(max(ak_stats),1.05*retro_stat))
ax.set_ylim(0, ymax)
ax.set_xlabel('Anderson Darling k-Sample Test Statistic')
ax.set_ylabel('Histogram Bin Count')
ax.set_title('Similarity Histogram: Anderson Darling k-Sample')
# ax[0].stairs(counts, bins, fill=True, alpha=.6)
# ax[0].plot([retro_stat_cross, retro_stat_cross], [0, ymax])
# ax[0].set_xlim(xstart, max(max(ak_stats_cross),1.05*retro_stat_cross))
# ax[0].set_ylim(0, ymax)
# ax[0].set_title('Cross-Similarity')

# RIGHT PLOT: histogram of AK(random_samples(COMBINED,len(RETRO))🠖COMBINED);
# orange line is AK(Retro🠖COMBINED)
# counts, bins = np.histogram(ak_stats_self)
# ymax = 1.02*max(counts)
# ax[1].stairs(counts, bins, fill=True, alpha=.6)
# ax[1].plot([retro_stat_self, retro_stat_self], [0, ymax])
# ax[1].set_xlim(xstart, max(max(ak_stats_self),1.05*retro_stat_self))
# ax[1].set_ylim(0, ymax)
# ax[1].set_title('Self-Similarity')


plt.show()

## Calculate final p-values

In [ ]:
# Set z-score for 99% confidence interval
    # NOTE: this is the confidence interval on the pvalues, which is necessary
    #   because we didn't we are sampling from the distribution instead of
    #   knowing it explicitly
z = 2.576

# How many test statistics for cross similarity of sub-samplings of the
#   combined dataset larger than test statistic for cross similarity of retro
pval = (ak_stats > retro_stat).mean()

# Calc confidence on self-similarity pval
conf = z*np.sqrt((pval*(1-pval))/N)

logfile_write(DATASET_FILENAME,[f'Similarity p-value: [{pval-conf:.3f},{pval+conf:.3f}]\n',f'  🠖 ~{pval*100:.2f}% chance that a sample could have been pulled which results in an AK\n\ttest statistic that is greater than or equal to the value seen when comparing\n\t(cosine similarities between Combined & itself) and\n\t(cosine similarities between Combined & Retro).'])
print(f'Similarity p-value: [{pval-conf:.3f},{pval+conf:.3f}]')
print(f'  🠖 ~{pval*100:.2f}% chance that a sample could have been pulled which results in an AK\n\ttest statistic that is greater than or equal to the value seen when comparing\n\t(cosine similarities between Combined & itself) and\n\t(cosine similarities between Combined & Retro).')

## Sanity Check with Coarse Metric

In [ ]:
# Calculate the mean cosine similarity value of subsets of embeddings
    # shorthand: mean(cosine_similarity(random_samples(COMBINED,len(RETRO))))
means = np.zeros(N,)
for i in range(N):
    means[i] = np.concatenate((cross_sim_vec(left_stack[i], pull_stack[i]),self_sim_mat(pull_stack[i]).compressed())).mean()

retro_mean = np.concatenate((cross_sim_vec(target_embs, retro_embs),self_sim_mat(retro_embs).compressed())).mean()

logfile_write(DATASET_FILENAME,[f'Mean Cosine Similarity for Retro: {retro_mean:.3f}'])
print(f'Mean Cosine Similarity for Retro: {retro_mean:.3f}')

In [ ]:
# Set z-score for 99% confidence interval
    # NOTE: this is the confidence interval on the pvalues, which is necessary
    #   because we didn't we are sampling from the distribution instead of
    #   knowing it explicitly
z = 2.576

# How many means of cosine similarities for sub-samplings of the
#   combined dataset are larger than the mean for Retro
pval = (means > retro_mean).mean()

# Calc confidence on pval
conf = z*np.sqrt((pval*(1-pval))/N)

logfile_write(DATASET_FILENAME,[f'Similarity p-value: [{pval-conf:.3f},{pval+conf:.3f}]\n',f'  🠖 ~{pval*100:.2f}% chance that a sample could have been pulled that has a mean\n\tcosine similarity which is greater than the mean cosine similarity of Retro.'])
print(f'Similarity p-value: [{pval-conf:.3f},{pval+conf:.3f}]')
print(f'  🠖 ~{pval*100:.2f}% chance that a sample could have been pulled that has a mean\n\tcosine similarity which is greater than the mean cosine similarity of Retro.')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(11,9))

# PLOT: histogram of mean(cosine_similarity(random_samples(COMBINED,len(RETRO))));
#    orange line is mean(cosine_similarity(Retro))
counts, bins = np.histogram(means)
ymax = 1.02*max(counts)
ax.stairs(counts, bins, fill=True, alpha=.6)
ax.plot([retro_mean, retro_mean], [0, ymax])
ax.set_xlim(min(min(bins),retro_mean), max(max(bins),1.05*retro_mean))
ax.set_ylim(0, ymax)
ax.set_xlabel('Mean Cosine Similarity')
ax.set_ylabel('Histogram Bin Count')
ax.set_title('Similarity Histogram: Mean Cosine Similarity')
ax.legend(['Mean cos-sim of pulls[i]', 'Mean cos-sim of Retro'])

plt.show()

In [ ]:
t1 = time.time()
print(f'Total time: {t1-t0:.1f}')